In [9]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/conda/lib/python3.8/site-packages/en_core_web_sm -->
/opt/conda/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
#https://towardsdatascience.com/k-means-clustering-chardonnay-reviews-using-scikit-learn-nltk-9df3c59527f3
 #   https://www.kaggle.com/venugopalvasarla/text-clustering-and-classification-with-nltk

In [10]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [11]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [12]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


In [18]:
import pandas as pd

articles = pd.read_csv("articles_text.csv")

In [56]:
text_data = []
for line in articles["text"]:
    if type(line) == type('str'):
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            text_data.append(tokens)
    else:
        text_data.append([])

['perez', 'basham', 'fisher', 'hobson', 'butler', 'white', 'anderson', 'racial', 'ethnic', 'disparity', 'covid-19', 'disease', 'burden', 'mortality', 'among', 'emergency', 'department', 'patient', 'safety', 'health', 'system', 'blumenthal', 'fowler', 'abrams', 'collins', 'covid-19-implications', 'health', 'system', 'black', 'emergency', 'physician', 'front', 'covid-19', 'ravage', 'community', 'disproportionately', 'killing', 'african', 'american', 'latino', 'minority', 'group', 'cheer', 'arrival', 'vaccine', 'along', 'front', 'provider', 'line', 'early', 'appointment', 'fill', 'pandemic', 'could', 'ndugga', 'artiga', 'parker', 'latest', 'covid-19', 'vaccination', 'ethnicity', 'walker', 'singhvi', 'holder', 'gebeloff', 'avila', 'pandemic', 'racial', 'disparity', 'persist', 'vaccine', 'rollout', 'times', 'warren', 'forrow', 'hodge', 'truog', 'trustworthiness', 'trust', 'covid-19', 'vaccine', 'trial', 'black', 'community', 'unfortunately', 'elation', 'short', 'live', 'early', 'sign', 'cur

In [59]:
len(articles["text"])

2939

In [60]:
len(text_data)

719

In [37]:
!pip install -U gensim

     |████████████████████████████████| 23.9 MB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 12.0 MB/s eta 0:00:01


In [39]:
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 9.3 MB/s  eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-linux_x86_64.whl size=184980 sha256=03ada6e289fcea037fd5a84005e3ef875114d2a6bc86335f32c8516ec64f710e
  Stored in directory: /root/.cache/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-Levenshtein


In [61]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [70]:
import gensim
NUM_TOPICS = 50
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=50)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=3)
for topic in topics:
    print(topic)

(7, '0.037*"moderna" + 0.025*"medicamentos" + 0.025*"basados"')
(49, '0.049*"emergency" + 0.037*"authorization" + 0.024*"covid-19"')
(11, '0.041*"index" + 0.041*"household" + 0.033*"patient"')
(37, '0.000*"vaccine" + 0.000*"covid-19" + 0.000*"medicine"')
(16, '0.031*"oxygen" + 0.028*"hospitalize" + 0.026*"patient"')
(25, '0.019*"challenge" + 0.019*"initiative" + 0.019*"others"')
(33, '0.016*"使用前，根据制造商说明存储所有检测部件" + 0.016*"准备采集样本" + 0.016*"如果您需要进行covid-19检测而无法获得医疗服务提供者检测，您可以考虑采用可在家或任何其他地方进行的"')
(1, '0.043*"please" + 0.035*"validate" + 0.035*"content"')
(12, '0.000*"covid-19" + 0.000*"family" + 0.000*"vaccine"')
(8, '0.000*"population" + 0.000*"older" + 0.000*"receive"')
(40, '0.000*"health" + 0.000*"covid-19" + 0.000*"symptom"')
(47, '0.023*"model" + 0.017*"cohort" + 0.014*"include"')
(29, '0.029*"symptom" + 0.022*"health" + 0.021*"covid"')
(0, '0.056*"population" + 0.040*"older" + 0.036*"years"')
(13, '0.032*"travel" + 0.032*"gathering" + 0.032*"family"')
(36, '0.000*"patient" + 0.000*"

In [80]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 20, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.042*"waiver" + 0.026*"request" + 0.023*"quarantine" + 0.023*"individual" + 0.022*"covid-19"')
(1, '0.035*"moderna" + 0.029*"website" + 0.012*"policy" + 0.012*"external" + 0.012*"privacy"')
(2, '0.036*"covid-19" + 0.026*"health" + 0.014*"pandemic" + 0.014*"global" + 0.012*"nurse"')
(3, '0.021*"health" + 0.020*"water" + 0.016*"global" + 0.015*"people" + 0.015*"sustainable"')
(4, '0.025*"oxygen" + 0.024*"hospitalize" + 0.020*"patient" + 0.020*"scale" + 0.017*"ordinal"')
(5, '0.024*"symptom" + 0.017*"health" + 0.017*"hopkins" + 0.017*"johns" + 0.014*"covid-19"')
(6, '0.044*"population" + 0.033*"older" + 0.032*"years" + 0.029*"receive" + 0.026*"total"')
(7, '0.001*"model" + 0.001*"admission" + 0.001*"cohort" + 0.001*"study" + 0.001*"mortality"')
(8, '0.030*"health" + 0.024*"public" + 0.024*"electronic" + 0.018*"report" + 0.018*"facility"')
(9, '0.053*"johnson" + 0.051*"vaccine" + 0.029*"covid-19" + 0.013*"country" + 0.011*"recommendation"')
(10, '0.034*"index" + 0.032*"household" + 0

In [83]:
articles["Resource URL"][2]

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'https://www.nejm.org/doi/10.1056/NEJMc2104974'

In [84]:
new_doc = prepare_text_for_lda(articles["text"][2])
new_doc_bow = dictionary.doc2bow(new_doc)
print(ldamodel.get_document_topics(new_doc_bow))

[(0, 0.044726446), (2, 0.060503278), (3, 0.029799256), (5, 0.027764), (6, 0.11787632), (9, 0.18739808), (10, 0.30234405), (11, 0.014066311), (14, 0.098716766), (16, 0.08384144), (19, 0.031252276)]


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
